In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
data = pd.read_csv('data/reviews.csv')

In [3]:
data = data[data['review'].apply(lambda x: isinstance(x, str))]
data = data[data['stars'].apply(lambda x: not np.isnan(x) and (x == 1 or x == 5))]
data['review'] = data['review'].apply(lambda x: ''.join(re.findall('\w+', x)))
data['created_at'] = data['created_at'].apply(pd.to_datetime)
data['year'] = data['created_at'].apply(lambda x: x.year)
data['stars'] = data['stars'].apply(lambda x: int(x))
data['stars_normalize'] = data['stars'].apply(lambda x: x / 5)

def star_to_binary(x):
    if x == 5:
        return 1
    elif x == 1:
        return 0
data['stars_binary'] = data['stars'].apply(lambda x: star_to_binary(x))

In [9]:
data['review']
symbols = []
for review in data['review']:
    symbols += list(review)
    
grouped_dict = {}    
for symbol in symbols:
    if symbol in grouped_dict:
        grouped_dict[symbol] += 1
    else: 
        grouped_dict[symbol] = 1
        
sorted_words = sorted([(k, v) for k, v in grouped_dict.items()], key=lambda x: x[1], reverse=True)
word2index = {}
index2word = {}
index2count = []
for idx, val in enumerate(sorted_words):
    index2word[idx] = val[0]
    word2index[val[0]] = idx
    index2count.append((idx, val[1]))
    
data['review_indexes'] = data['review'].apply(lambda x: list(map(lambda word: word2index[word], list(x))))

In [24]:
x_train = data[data['year'] < 2018]['review_indexes']
x_test = data[data['year'] == 2018]['review_indexes']

y_train = data[data['year'] < 2018]['stars_binary']
y_train_normalize = data[data['year'] < 2018]['stars_normalize']
y_train_binary = data[data['year'] < 2018]['stars_binary']

y_test_binary = data[data['year'] == 2018]['stars_binary']
y_test_normalize = data[data['year'] == 2018]['stars_normalize']
y_test = data[data['year'] == 2018]['stars_binary']

In [26]:
print('Maximum review length: {}'.format(len(max((pd.Series(x_train).append(pd.Series(x_test))), key=len))))
print('Minimum review length: {}'.format(len(min((pd.Series(x_train).append(pd.Series(x_test))), key=len))))

# the histogram of the data
n, bins, patches = plt.hist(list(map(lambda x: word2index[x], symbols)), 1000)
plt.grid(True)
plt.show()

Maximum review length: 727
Minimum review length: 0


In [28]:
from keras import Sequential
from keras.layers import Dropout, Activation, Embedding, LSTM, Dense
from keras.preprocessing import sequence

max_words = 500
x_train = sequence.pad_sequences(x_train, maxlen=max_words)
x_test = sequence.pad_sequences(x_test, maxlen=max_words)

model = Sequential()
model.add(Embedding(len(grouped_dict), 128, input_length=max_words))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 500, 128)          505088    
_________________________________________________________________
lstm_9 (LSTM)                (None, 500, 64)           49408     
_________________________________________________________________
lstm_10 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
_________________________________________________________________
activation_5 (Activation)    (None, 1)                 0         
Total params: 587,585
Trainable params: 587,585
Non-trainable params: 0
_________________________________________________________________
None

In [ ]:
model.fit(x_train, y_train)
model.save('models/jd_reviews_model.h5')

NameError: name 'model' is not defined

In [1]:
accuracy = model.evaluate(x_test, y_test)[1]
print(f"Predict with accuracy {accuracy}%")

Xtest {x_test[:10]}


NameError: name 'y_test' is not defined